In [ ]:
def sudoku(input_list, r_group, c_group, b_group):

    result_list = []

    def check_availbable(r, c, b):
        if(len(input_list)<0):
            return True

        for i in range(9):
            if(r_group[r][i] and c_group[c][i] and b_group[b][i]):

                r_group[r][i] = False
                c_group[c][i] = False
                b_group[b][i] = False
                input_list.pop()

                if(check_availbable(**input_list[-1])):
                    result_list.append((r, c, i))
                    return True

                r_group[r][i] = True
                c_group[c][i] = True
                b_group[b][i] = True
                input_list.append((r, c, b))

    return result_list

                




r_group = [[True for _ in range(9)] for _ in range(9)]
c_group = [[True for _ in range(9)] for _ in range(9)]
b_group = [[True for _ in range(9)] for _ in range(9)]

result_matrix = [[0 for _ in range(9)] for _ in range(9)]
input_list = []

for r in range(9):
    for c, n in enumerate(map(int, input().split())):
        b = 3 * r//3 + c//3

        result_matrix[r][c] = n
        if(n==0):
            input_list.append((r, c, b))
        else:
            r_group[r][n-1] = False
            c_group[c][n-1] = False
            b_group[b][n-1] = False

results = sudoku(input_list, r_group, c_group, b_group)

for result in results:
    r, c, n = result

    result_matrix[r][c] = n

for r in range(result_matrix):
    print(" ".join(map(str, r)))



In [ ]:
import sys
sys.setrecursionlimit(10000)

row_starts = []

class node:

    def __init__(self):

        self.row = 0
        self.num = (-1, -1, -1)
        self.col = 0

        self.column = False
        self.up = False
        self.down = False
        self.left = False
        self.right = False

    def skip(self):
        
        current = row_starts[self.row]

        while(current):
            current.skip_down()
            current.skip_col()

            current = current.right

    def recover(self):

        current = row_starts[self.row]

        while(current):
            current.recover_down()
            current.recover_col()

            current = current.right

    def skip_row(self):
        ## 해당 row를 제거

        current = row_starts[self.row]

        while(current):
            if(current.up):
                current.up.down = current.down
            if(current.down):
                current.down.up = current.up

            current = current.right

    def recover_row(self):
        current = row_starts[self.row]

        while(current):
            if(current.up):
                current.up.down = current
            if(current.down):
                current.down.up = current

            current = current.right

    def skip_down(self):

        # self.set_col()
        self.skip_row()

        if(self.down):
            self.down.skip_down()

    def recover_down(self):

        # self.set_col()
        self.recover_row()

        if(self.down):
            self.down.recover_down()

    def skip_col(self):

        if(self.column.left):
            self.column.left.right = self.column.right
        if(self.column.right):
            self.column.right.left = self.column.left

    def recover_col(self):
        if(self.column.right):
            self.column.right.left = self.column
        if(self.column.left):
            self.column.left.right = self.column



def solve():

    correct_list = []

    def search(head):
        if(not head.right):
            return True
        
        next_node = head.right
        while(next_node.down):
            # print(next_node.down.num)
            next_node = next_node.down
            next_node.skip()
            correct_list.append(next_node.row)

            if(search(head)):
                return True
            correct_list.pop()

            next_node.recover()
        
        return False
            
    columns = []
    #0~80(r, c) : r, c에 수가 들어간다 -> 9*(r-1) + (c-1)
    #81~161(r, n) : r에 숫자 n가 들어간다 -> 81 + 9*(r-1) + (n-1)
    #162 ~ 242(c, n) : c에 숫자 n가 들어간다 -> 162 + 9*(c-1) + (n-1)
    #243 ~ 323(b, n) : b에 숫자 n가 들어간다 -> 243 + 9(b-1) + (n-1)


    ##initialize cols
    previous_col = False
    for i in range(9*9*4):
        col = node()
        col.col = i

        if(previous_col):
            previous_col.right = col
        col.left = previous_col
        previous_col = col

        columns.append(col)
    
    up_col = columns.copy()

    #initialize rows
    row_idx = -1
    for r in range(9):
        for c, n in enumerate(map(int, input().split())):
            b = r//3 * 3 + c//3
            if(n==0):
                for init_n in range(9):
                    previous = False
                    row_idx += 1
                    for col_idx in (9*r+c, 81 + 9*r+init_n, 162 + 9*c+init_n, 243 + 9*b+init_n):
                        row_node = node()
                        row_node.row = row_idx
                        row_node.num = (r, c, init_n)

                        if(previous):
                            previous.right = row_node
                        else:
                            # print(row_node.num)
                            row_starts.append(row_node)
                        
                        row_node.left = previous
                        previous = row_node
                        row_node.up = up_col[col_idx]
                        up_col[col_idx].down = row_node
                        up_col[col_idx] = row_node

                        row_node.column = columns[col_idx]
            else:
                row_idx += 1
                init_n = n-1
                previous = False
                for col_idx in (9*r+c, 81 + 9*r+init_n, 162 + 9*c+init_n, 243 + 9*b+init_n):
                    row_node = node()
                    row_node.row = row_idx
                    row_node.num = (r, c, init_n)

                    if(previous):
                        previous.right = row_node
                    else:
                        # print(row_node.num)
                        row_starts.append(row_node)
                    
                    row_node.left = previous
                    previous = row_node

                    row_node.up = up_col[col_idx]
                    up_col[col_idx].down = row_node
                    up_col[col_idx] = row_node

                    row_node.column = columns[col_idx]

    head = node()
    head.right = columns[0]
    columns[0].left = head
    search(head)
    return correct_list


matrix = [[0 for _ in range(9)] for _ in range(9)]
corrects = solve()
col_nums = 0
for correct_node in corrects:
    # print(correct_node)
    r, c, n = row_starts[correct_node].num

    matrix[r][c] = n+1

for res in matrix:
    print(" ".join(map(str, res)))



